# Computing WACC and Terminal Growth Rate

## Linear Regression to find Beta

In [132]:
import yfinance as yf
import pandas as pd
import numpy as np

Returns_frame=pd.DataFrame()

tickers=["LT.NS"]
assets=len(tickers)

risk_free_rate=0.065

def import_data(ticker_name):
  data = yf.download(ticker_name, start='2022-03-31', end='2025-11-01', interval='1d')
  daily_returns = np.log(data['Close']/data['Close'].shift(1))
  return (daily_returns)

for ticker in tickers:
  Returns_frame[ticker]=import_data(ticker)

Returns_frame = Returns_frame.dropna()
print(Returns_frame.head)

C:\Users\nisha\AppData\Local\Temp\ipykernel_12256\861138211.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker_name, start='2022-03-31', end='2025-11-01', interval='1d')
[*********************100%***********************]  1 of 1 completed

<bound method NDFrame.head of                LT.NS
Date                
2022-04-01  0.012620
2022-04-04  0.020048
2022-04-05  0.005297
2022-04-06  0.009082
2022-04-07 -0.014406
...              ...
2025-10-27  0.004828
2025-10-28  0.012411
2025-10-29 -0.003707
2025-10-30  0.007400
2025-10-31  0.010825

[886 rows x 1 columns]>


In [133]:
Returns_frame["NIFTY"]=import_data("^NSEI")
import pandas as pd
import statsmodels.api as sm
beta=0

def linear_regress(stock, index):

    rf_daily = (1 + risk_free_rate) ** (1/252) - 1

    Y_excess = (index - rf_daily).rename("NIFTY_Excess_Ret")
    X_excess = (stock - rf_daily).rename("Stock_Excess_Ret")

    data = pd.concat([X_excess, Y_excess], axis=1).dropna()

    Y = data["NIFTY_Excess_Ret"]
    X = data["Stock_Excess_Ret"]

    X_with_const = sm.add_constant(X)

    model = sm.OLS(Y, X_with_const)
    results = model.fit()

    return(results)


for stock in tickers:
    x = Returns_frame[stock]
    y = Returns_frame["NIFTY"]
    print("The Regression Results for", stock, "are:")
    print("\nFull OLS Summary for:",stock)
    results=linear_regress(x, y)
    print(results.summary())
    print("-" * 40)
    print(" ")
    beta=(results.params['Stock_Excess_Ret'])
    
print(beta)

C:\Users\nisha\AppData\Local\Temp\ipykernel_12256\861138211.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker_name, start='2022-03-31', end='2025-11-01', interval='1d')
[*********************100%***********************]  1 of 1 completed

The Regression Results for LT.NS are:

Full OLS Summary for: LT.NS
                            OLS Regression Results                            
Dep. Variable:       NIFTY_Excess_Ret   R-squared:                       0.403
Model:                            OLS   Adj. R-squared:                  0.403
Method:                 Least Squares   F-statistic:                     597.2
Date:                Fri, 21 Nov 2025   Prob (F-statistic):          3.53e-101
Time:                        07:36:28   Log-Likelihood:                 3227.4
No. Observations:                 886   AIC:                            -6451.
Df Residuals:                     884   BIC:                            -6441.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------

In [134]:
daily_market_return=Returns_frame["NIFTY"].mean()
annualized_market_return=np.power((1+daily_market_return),252)-1
print(annualized_market_return)

E_r=risk_free_rate+(beta*(annualized_market_return-risk_free_rate))
print("The Equity Cost of Capital for",tickers,"is:",E_r*100,"%")

0.11638455319451069
The Equity Cost of Capital for ['LT.NS'] is: 8.276669878239321 %


## Cost of Debt Computation

In [135]:
#First we compute the Interest Coverage Ratio for the firm
folder_path=r"C:\Users\nisha\Desktop\2026 PS-II\Offshoot courses\ECON F355 - BAV\Assignment\pythonProject\data"
FCFF=pd.read_csv(folder_path+'\\'+"FCFF.csv",index_col=0)
FCFE=pd.read_csv(folder_path+'\\'+"FCFE.csv",index_col=0)

FCFE=FCFE.T
FCFF=FCFF.T

ICR=pd.DataFrame()
ICR["Interest"]=FCFE["interest expense"]
ICR["EBIT"]=FCFF["EBIT"]
ICR["ICR"]=ICR["EBIT"]/ICR["Interest"]
ICR=ICR.T

for i in range(1,6):
    a="Year"+str(i)
    ICR.drop(a, axis=1, inplace=True)
    
print(ICR)


           Base Year
Interest   23.621883
EBIT      190.839587
ICR         8.078932


In [136]:
#ICR lies between 6.5 and 8.5 so the rating is AA and default spread is 0.60%
cost_of_debt=risk_free_rate+(0.60/100)
print(cost_of_debt)

0.07100000000000001


## Computing WACC

In [137]:
tax=0.3
shares_outstanding=1375357414
ticker=yf.Ticker('LT.NS')
price_per_share=ticker.fast_info.last_price
E=shares_outstanding*price_per_share/(np.power(10,10))
bs=pd.read_csv(folder_path+'\\'+'Balance_sheet_req_data.csv',index_col=0)
D=bs.iloc[0,0]
V=E+D
print(V)

4770.4108269077915


In [138]:
WACC=E_r*(E/V)+cost_of_debt*(1-tax)*(D/V)
print(str(WACC*100)+"%")

7.699688812282784%


## Computing Terminal Growth Rate

In [139]:
income_statement=pd.read_csv(folder_path+'\\'+'Income_statement_req_data.csv',index_col=0)
net_profit=income_statement.iloc[8,0]

print(net_profit)

ROE=net_profit/E
print(ROE)

div=pd.DataFrame()
div=ticker.dividends
div_1=(div.iloc[-1])

dividend_yield=(div_1/price_per_share)
print(dividend_yield)

terminal_growth_rate=(ROE*(1-dividend_yield))

ICR=ICR.T
ICR["Debt"]=D
ICR["Equity"]=E
ICR["Value"]=V
ICR["Cost of Debt"]=cost_of_debt
ICR["Cost of Equity"]=E_r*100
ICR["WACC"]=WACC
ICR["div"]=div_1
ICR["ROE"]=ROE
ICR["Dividend yield"]=dividend_yield
ICR["terminal_growth_rate"]=terminal_growth_rate

print(str(terminal_growth_rate*100)+"%")
ICR.drop(ICR.iloc[0:-1].index, inplace=True)
ICR.index=["Base Year"]
ICR=ICR.T
print(ICR)
ICR.to_csv(folder_path+'\\'+"WACC.csv")

106.6412232701194
0.027079898712997416
0.008421261411400607
2.685185180694101%
                        Base Year
Interest                23.621883
EBIT                   190.839587
ICR                      8.078932
Debt                   832.389330
Equity                3938.021497
Value                 4770.410827
Cost of Debt             0.071000
Cost of Equity           8.276670
WACC                     0.076997
div                     34.000000
ROE                      0.027080
Dividend yield           0.008421
terminal_growth_rate     0.026852
